In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import torch
torch.cuda.is_available()

True

In [ ]:
!nvidia-smi

Thu Mar 21 19:18:20 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   57C    P8              10W /  70W |      3MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [ ]:
!pip install scikit-ntk

In [ ]:
import tensorflow as tfa
import keras
from keras.layers import Dense,Flatten,MaxPool2D,Conv2D,Dropout,Activation,BatchNormalization
from keras.models import Sequential
from skntk import NeuralTangentKernel as NTK
import numpy as np
import matplotlib.pyplot as plt
import tensorflow.signal as tf
print("all the libraries are imported")

all the libraries are imported


In [ ]:
dataset_dir=r'/content/drive/MyDrive/the dataset/dataset copy/Copy of data'
image_height=512
image_width=512
batch_size=32
seed=42
validation_split=0.1

In [ ]:
train=keras.preprocessing.image_dataset_from_directory(
    dataset_dir,
    batch_size=batch_size,
    label_mode='int',
    image_size=(image_height,image_width),
    seed=seed,
    validation_split=validation_split,
    subset='training'
)
test=keras.preprocessing.image_dataset_from_directory(
    dataset_dir,
    label_mode='int',
    batch_size=batch_size,
    image_size=(image_height,image_width),
    seed=seed,
    validation_split=validation_split,
    subset='validation'
)

Found 1564 files belonging to 2 classes.
Using 1408 files for training.
Found 1564 files belonging to 2 classes.
Using 156 files for validation.


In [ ]:
input_shape=(image_height,image_width,3)

In [ ]:
import tensorflow as tf

print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

Num GPUs Available:  1


In [ ]:
resize_rescale=keras.Sequential([
    keras.layers.Resizing(image_height,image_width),
    keras.layers.Rescaling(1./255)
])

In [ ]:
data_augmentation=keras.Sequential([
    keras.layers.RandomFlip("horizontal_and_vertical"),
    keras.layers.RandomRotation(0.2),
])

In [ ]:
with tf.device('/GPU:0'):
  def fourier(inp=None):
    if inp==None:
        pass
    elif(inp!=None):
        return tf.fft3d(inp)

  def ntk(inp=None):
    if(inp==None):
        pass
    elif(inp!=None):
        n = NTK(depth=3, bias=0.01, bias_bounds=(1e-5, 1e5))
        return n
  model=Sequential([
    resize_rescale,
    data_augmentation,
    Conv2D(64,(2,2),padding='valid',input_shape=input_shape),
    Activation(ntk()),
    Dropout(0.2),
    MaxPool2D(pool_size=(2,2)),

    Conv2D(64,(2,2),padding='valid'),
    Activation(ntk()),
    Dropout(0.2),
    MaxPool2D(pool_size=(2,2)),

    Conv2D(64,(2,2),padding='valid'),
    Activation(ntk()),
    Dropout(0.2),
    MaxPool2D(pool_size=(2,2)),

    Conv2D(64,(2,2),padding="valid"),
    Activation(ntk()),
    Dropout(0.2),
    MaxPool2D(pool_size=(2,2)),

    Flatten(),

    Dense(128),
    Activation(fourier()),
    BatchNormalization(),

    Dense(128),
    Activation(fourier()),
    BatchNormalization(),

    Dense(128),
    Activation(fourier()),

    Dense(1),
    Activation('sigmoid')
  ])
  model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
  history=model.fit(train,epochs=150,batch_size=32)

Epoch 1/150
44/44 [==============================] - 226s 4s/step - loss: 0.6496 - accuracy: 0.7088
Epoch 2/150
44/44 [==============================] - 43s 951ms/step - loss: 0.5513 - accuracy: 0.7500
Epoch 3/150
44/44 [==============================] - 43s 953ms/step - loss: 0.5033 - accuracy: 0.7791
Epoch 4/150
44/44 [==============================] - 44s 951ms/step - loss: 0.4987 - accuracy: 0.7884


In [ ]:
model.summary()

In [ ]:
accuracy=model.evaluate(test)
print(accuracy)

In [ ]:
plt.plot(history.history['accuracy'])
plt.title('model accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

In [ ]:
amcuracy=model.evaluate(valid)
print(amcuracy)